In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [5]:
def conv_block(in_list,out_list,k_list,p_list,pooling_k, pooling_s):
    conv_list=[] 
    for i in range(len(in_list)):
        conv = nn.Sequential(
            nn.Conv2d(in_list[i],out_list[i],kernel_size=k_list[i],padding=p_list[i]),
            nn.ReLU()
        )
        conv_list.append(conv)
    conv_list.append(nn.MaxPool2d(pooling_k,pooling_s))
        
    return nn.Sequential(*conv_list)    

In [6]:
def fc_block(in_size,out_size):
    fc = nn.Sequential(
        nn.Linear(in_size,out_size),
        nn.ReLU()
    )
    
    return fc

In [28]:
class VGG(nn.Module):
    def __init__(self,num_classes):
        super(VGG,self).__init__()
        
        self.conv1 = conv_block([3,64],[64,64],[3,3],[1,1],2,2)
        self.conv2 = conv_block([64,128],[128,128],[3,3],[1,1],2,2)
        self.conv3 = conv_block([128,256,256],[256,256,256],[3,3,3],[1,1,2],2,2)
        self.conv4 = conv_block([256,512,512],[512,512,512],[3,3,3],[1,1,1],2,2)
        self.conv5 = conv_block([512,512,512],[512,512,512],[3,3,3],[1,1,1],2,2)
        
        self.fc1 = fc_block(7*7*512,4096)
        self.fc2 = fc_block(4096,4096)
        self.out_fc = fc_block(4096,num_classes)
        
    def forward(self,x):
        
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        
        x = torch.flatten(x)
        #x = x.view(x.size(0),-1)
        
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.out_fc(x)
        
        return x

In [29]:
model = VGG(num_classes=16)

In [30]:
model

VGG(
  (conv1): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU()
    )
    (1): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU()
    )
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Sequential(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU()
    )
    (1): Sequential(
      (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU()
    )
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv3): Sequential(
    (0): Sequential(
      (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU()
    )
    (1): Sequential(
      (0): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU()
    )
    (2): Seq